# Project 6: IMDB

This project involves NLP, decision trees, bagging, boosting, and more!

---

## Load packages

You are likely going to need to install the `imdbpie` package:

    > pip install imdbpie

---

In [1]:
import os
import subprocess
import collections
import re
import csv
import json

import pandas as pd
import numpy as np
import scipy

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import psycopg2
import requests
from imdbpie import Imdb
import nltk

import urllib
from bs4 import BeautifulSoup
import nltk

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

---

## Part 1: Acquire the Data

You will connect to the IMDB API to query for movies. 

See here for documentation on how to use the package:

https://github.com/richardasaurus/imdb-pie

#### 1. Connect to the IMDB API

In [2]:
imdb = Imdb()
imdb = Imdb(anonymize=True)
imdb = Imdb(cache=True)

#### 2. Query the top 250 rated movies in the database

In [3]:
tp250 = imdb.top_250()
#[{'title': 'The Shawshank Redemption', 'year': '1994', 'type': 'feature', 'rating': 9.3,...}, ...]

#### 3. Make a dataframe from the movie data

Keep the fields:

    num_votes
    rating
    tconst
    title
    year
    
And discard the rest

In [4]:
df = pd.DataFrame(tp250)

In [5]:
df.head()

,can_rate,image,num_votes,rating,tconst,title,type,year
0,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1659650,9.3,tt0111161,The Shawshank Redemption,feature,1994
1,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1135992,9.2,tt0068646,The Godfather,feature,1972
2,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,775849,9.0,tt0071562,The Godfather: Part II,feature,1974
3,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1645344,9.0,tt0468569,The Dark Knight,feature,2008
4,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,849066,8.9,tt0108052,Schindler's List,feature,1993


# Delete columns as specified

In [6]:
#del column 
# del(deletes)
#dataframe name and name of column in brackets
del df["can_rate"]

In [7]:
del df["image"]

In [8]:
del df["type"]

In [9]:
df.dtypes

num_votes      int64
rating       float64
tconst        object
title         object
year          object
dtype: object

# Convert year to integer

In [10]:
df["year"] = df["year"].astype(int)

# check unique values in columns for oddities

In [11]:
for i in df:
    print i
    print df[i].unique(),

num_votes
[1659650 1135992  775849 1645344  849066  437763 1300701 1194329  494757
 1321370 1217791  821075 1224588 1438914 1080496  668490  715454 1195945
  225349  894688  524531 1003873  869112  271004  727765  410794  711735
  212192  407925  864288  769579  898494  381588  414203  105727  649819
  490435  310415  139681  766062  732119  488024  857221  716684  381669
  965101  831086  443692  826433  629719  337003  137021  553404  134454
  255003  149129  933567  597984  118583  134540  702456 1122878  808988
  480682  216768  348814  293390  216999  217031  165618  243861  667524
   61422   97373  651864  525010  724509  563422  547452  496856   95782
  532589  210810  189319  654012  475630  266023  172420  418988   92454
  147339  591448  609372  376183  866240   65371  408118  153805  107261
  100620  506211  528007  148877  109801   81547   72631   25463   77816
  962161  164969  200579  172189  592594  272889  649396  111790  231508
  320261  244670  583042  162779  212601 

#### 3. Select only the top 100 movies

In [12]:
#sort dataframes by number of votes in descending order
df100 = df.sort_index(by='num_votes', ascending= False)

/Users/ltdanp21/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  from ipykernel import kernelapp as app


In [13]:
# check num of votes at 1st and one hundredth spot
df100.head(101)

,num_votes,rating,tconst,title,year
0,1659650,9.3,tt0111161,The Shawshank Redemption,1994
3,1645344,9.0,tt0468569,The Dark Knight,2008
13,1438914,8.8,tt1375666,Inception,2010
9,1321370,8.9,tt0137523,Fight Club,1999
6,1300701,8.9,tt0110912,Pulp Fiction,1994
12,1224588,8.8,tt0109830,Forrest Gump,1994
10,1217791,8.8,tt0120737,The Lord of the Rings: The Fellowship of the Ring,2001
17,1195945,8.7,tt0133093,The Matrix,1999
7,1194329,8.9,tt0167260,The Lord of the Rings: The Return of the King,2003
1,1135992,9.2,tt0068646,The Godfather,1972


In [14]:
# set mask greater than and equal number of votes at 100th position
mask100 = df100["num_votes"] >= 437656

In [15]:
#create new dataframe equal to top 100 movies
top100 = df[mask100]

/Users/ltdanp21/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


#### 4. Get the genres and runtime for each movie and add them to the dataframe

There can be multiple genres per movie, so this will need some finessing.

In [16]:
# collect genre from API
genre = []

for i in top100["tconst"]:
    a = imdb.get_title_by_id(i)
    genre.append(a.genres)

print len(genre)

100


In [17]:
#add column to top 100 dataframe
top100["genre"] = genre

/Users/ltdanp21/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [18]:
#extract runtime via api and append to list
runtime = []

for t in top100["tconst"]:
    r = imdb.get_title_by_id(t)
    runtime.append(r.runtime)
    
print runtime
    

[8520, 10500, 12120, 9120, 11700, 5760, 9240, 12060, 9660, 8340, 10680, 7440, 8520, 8880, 10740, 7980, 8760, 8160, 7260, 7800, 7620, 7080, 6360, 6600, 10140, 6060, 10140, 6900, 6720, 11340, 9180, 9000, 9060, 6960, 9300, 6780, 9180, 7800, 5340, 7020, 9900, 8640, 5880, 9840, 7320, 9240, 7860, 5940, 7320, 10680, 6120, 8160, 6780, 6180, 6480, 6960, 6120, 4860, 9180, 7620, 10200, 8400, 7560, 5760, 7860, 7080, 8160, 7020, 10800, 8100, 7920, 6960, 7020, 5880, 5640, 5880, 6420, 6000, 7320, 8940, 6660, 7200, 8280, 5940, 7920, 7620, 6900, 6780, 6180, 6420, 5520, 7800, 7740, 8820, 6840, 8460, 8580, 8580, 7260, 7080]


In [19]:
#add runtimes to dataframe
top100["runtime"] = runtime

/Users/ltdanp21/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [111]:
top100.title.iloc[0:5].values

array([u'The Shawshank Redemption', u'The Godfather',
       u'The Godfather: Part II', u'The Dark Knight', u"Schindler's List"], dtype=object)

In [21]:
top100.year.values

array([1994, 1972, 1974, 2008, 1993, 1957, 1994, 2003, 1966, 1999, 2001,
       1980, 1994, 2010, 2002, 1975, 1990, 1999, 1977, 2002, 1995, 1991,
       1995, 1994, 1998, 1998, 2014, 1981, 2011, 1999, 1991, 2002, 2006,
       1985, 2000, 2000, 1979, 2006, 1994, 1979, 2012, 1980, 2008, 2012,
       1999, 1986, 1983, 1992, 2001, 1995, 2000, 1971, 1976, 2010, 2004,
       1987, 2000, 1995, 2009, 1989, 1983, 2005, 1997, 2009, 1988, 2006,
       2015, 1982, 2013, 2001, 2005, 2008, 1998, 1996, 1996, 2010, 1999,
       2003, 2007, 2014, 2003, 2015, 2010, 2014, 2004, 1993, 2007, 2001,
       1998, 1984, 2001, 2011, 1995, 2005, 2014, 2002, 2003, 2012, 2014,
       2010])

#### 4. Write the Results to a csv

In [22]:
# write dataframe to csv
movies_csv = top100.to_csv

---

## Part 2: Wrangle the text data

#### 1. Scrape the reviews for the top 100 movies

*Hint*: Use a loop to scrape each page at once

In [23]:
import urllib2
from bs4 import BeautifulSoup 

In [99]:
# unique ids for top 100 movies
tconst = top100["tconst"].values
tconstsmall = top100["tconst"].iloc[0:5].values
start = range(0, 250, 10)

In [40]:
import time

In [100]:
tconstsmall

array([u'tt0111161', u'tt0068646', u'tt0071562', u'tt0468569', u'tt0108052'], dtype=object)

# I got stuck trying to get my scarping function to work. At first I tried to use the string formatters to autopopulate the elements of the url that needed to change inside the urlopen fucntion, but that did not work. To get my function to work I assigned the URL to a variable, and the passed the variable to the URL open function.

# Next I attempted to scarpe all the 250 reviews for each move in the top 100. After three hours the operation was not done running so I cut it off and proceeded with what I had. In the process of manipulating the data for TfidfVectorizer analysis, I accidentally overrid the list of reviews I created. In the interest of time I scarped 250 reviews for the top five movies only. 



#### 2. Extract the reviews and the rating per review for each movie

*Note*: "soup" from BeautifulSoup is the html returned from all 25 pages. You'll need to either address each page individually or break them down by elements.

In [101]:
reviews = [] 



def get_reviews(tcon,strt):
    variable = "http://www.imdb.com/title/%s/reviews?start=%d" % (tcon,strt)
    response = urllib2.urlopen(variable)  
    soup = BeautifulSoup(response)
    comment_parent = soup.find(id='tn15content')
    p_tags = comment_parent.find_all('p')

    for p in p_tags:
        text = p.get_text()
        if text != '*** This review may contain spoilers ***' and text != 'Add another review':
            reviews.append([text])
    
    return reviews

for t in tconstsmall:
    time.sleep(.02)
    for s in range(0, 50, 10):
        get_reviews(t,s)

In [104]:
import copy
new_list = copy.copy(reviews)

#### 3. Remove the non AlphaNumeric characters from reviews

In [117]:
newdf = pd.DataFrame(new_list, columns = ['reviews'])

# I also got stuck here. I did not realize setting max features to 200 would ensure the return of the top 200 hundred words only. I lost hours trying to figure out how to sort the dataframe (which of course is not possible) to see which words were the top 200. UG FAIL! 

#### 4. Calculate the top 200 ngrams from the user reviews

Use the `TfidfVectorizer` in sklearn.

Recommended parameters:

    ngram_range = (1, 2)
    stop_words = 'english'
    binary = False
    max_features = 200

In [148]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvec = TfidfVectorizer(ngram_range = (1, 2),stop_words = 'english',binary = False, max_features = 200)
m = tvec.fit_transform(newdf["reviews"])

In [149]:
n = m.todense()

In [169]:
ngram = pd.DataFrame(n,columns = tvec.get_feature_names())

In [189]:
tvec.get_feature_names()

[u'10',
 u'acting',
 u'action',
 u'actor',
 u'actors',
 u'actually',
 u'al',
 u'al pacino',
 u'amazing',
 u'andy',
 u'away',
 u'bad',
 u'bale',
 u'based',
 u'batman',
 u'begins',
 u'believe',
 u'best',
 u'better',
 u'bit',
 u'black',
 u'book',
 u'brando',
 u'brilliant',
 u'business',
 u'caan',
 u'cast',
 u'cazale',
 u'character',
 u'characters',
 u'cinematography',
 u'come',
 u'comes',
 u'coppola',
 u'corleone',
 u'crime',
 u'dark',
 u'dark knight',
 u'day',
 u'did',
 u'didn',
 u'different',
 u'direction',
 u'director',
 u'does',
 u'doesn',
 u'don',
 u'duvall',
 u'end',
 u'especially',
 u'evil',
 u'excellent',
 u'experience',
 u'face',
 u'fact',
 u'family',
 u'far',
 u'father',
 u'feel',
 u'film',
 u'films',
 u'ford',
 u'ford coppola',
 u'francis',
 u'francis ford',
 u'fredo',
 u'freeman',
 u'gangster',
 u'gets',
 u'gives',
 u'godfather',
 u'godfather ii',
 u'goes',
 u'going',
 u'good',
 u'got',
 u'great',
 u'greatest',
 u'having',
 u'head',
 u'heath',
 u'heath ledger',
 u'high',
 u'hi

#### 5. Merge the user reviews and ratings

#### 6. Save this merged dataframe as a csv

---

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

#### 2. Connect to database and query the joined set

#### 3. Join the two tables 

#### 4. Select the newly joined table and save two copies of the into dataframes

---

## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename any columns you think should be renamed for clarity

#### 2. Describe anything interesting or suspicious about your data (quality assurance)

#### 3. Make four visualizations of interest to you using the data

---

## Part 5: Decision Tree Classifiers and Regressors

#### 1. What is our target attribute? 

Choose a target variable for the decision tree regressor and the classifier. 

#### 2. Prepare the X and Y matrices and preprocess data as you see fit

#### 3. Build and cross-validate your decision tree classifier

#### 4. Gridsearch optimal parameters for your classifier. Does the performance improve?

#### 5. Build and cross-validate your decision tree regressor

#### 6. Gridsearch the optimal parameters for your classifier. Does performance improve?

---

## Part 6: Elastic Net


#### 1. Gridsearch optimal parameters for an ElasticNet using the regression target and predictors you used for the decision tree regressor.


#### 2. Is cross-validated performance better or worse than with the decision trees? 

#### 3. Explain why the elastic net may have performed best at that particular l1_ratio and alpha

---

## Part 7: Bagging and Boosting: Random Forests, Extra Trees, and AdaBoost

#### 1. Load the random forest regressor, extra trees regressor, and adaboost regressor from sklearn

#### 2. Gridsearch optimal parameters for the three different ensemble methods.

#### 3. Evaluate the performance of the two bagging and one boosting model. Which performs best?

#### 4. Extract the feature importances from the Random Forest regressor and make a DataFrame pairing variable names with their variable importances.

#### 5. Plot the ranked feature importances.

#### 6.1 [BONUS] Gridsearch an optimal Lasso model and use it for variable selection (make a new predictor matrix with only the variables not zeroed out by the Lasso). 

#### 6.2 [BONUS] Gridsearch your best performing bagging/boosting model from above with the features retained after the Lasso. Does the score improve?

#### 7.1. [BONUS] Select a threshold for variable importance from your Random Forest regressor and use that to perform feature selection, creating a new subset predictor matrix.

#### 7.2 [BONUS] Using BaggingRegressor with a base estimator of your choice, test a model using the feature-selected dataset you made in 7.1

---

## [VERY BONUS] Part 8: PCA

#### 1. Perform a PCA on your predictor matrix

#### 2. Examine the variance explained and determine what components you want to keep based on them.

#### 3. Plot the cumulative variance explained by the ordered principal components.

#### 4. Gridsearch an elastic net using the principal components you selected as your predictors. Does this perform better than the elastic net you fit earlier?

#### 5. Gridsearch a bagging ensemble estimator that you fit before, this time using the principal components as predictors. Does this perform better or worse than the original? 

#### 6. Look at the loadings of the original predictor columns on the first 3 principal components. Is there any kind of intuitive meaning here?

Hint, you will probably want to sort by absolute value of magnitude of loading, and also only look at the obviously important (larger) ones!

# [Extremely Bonus] Part 9:  Clustering

![](https://snag.gy/jPSZ6U.jpg)

 ***Bonus Bonus:***
This extended bonus question is asking to do something we never really talked about but would like for you to attempt based on the assumptions that we learned during this weeks clustering lesson(s).

#### 1. Import your favorite clustering module

#### 2. Encode categoricals

#### 3. Evaluate cluster metics solely based on a range of K
If K-Means:  SSE/Inertia vs Silhouette (ie: Elbow), silhouette average, etc

#### 4.  Look at your data based on the subset of your predicted clusters.
Assign the cluster predictions back to your dataframe in order to see them in context.  This is great to be able to group by cluster to get a sense of the data that clumped together.

#### 5. Describe your findings based on the predicted clusters 
_How well did it do?  What's good or bad?  How would you improve this? Does any of it make sense?_